In [1]:
import os
import sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import SimpleITK as sitk
import skimage
from os.path import expanduser
import glob
from matplotlib.ticker import MaxNLocator
from scipy.stats import spearmanr
import seg_metrics.seg_metrics as sg
import pickle
import matplotlib.patches as mpatches
from skimage import exposure
from scipy import ndimage
import nibabel as nib

In [2]:
print("Python version:", sys.version)
print("Numpy version:", np.__version__)
print("Matplotlib version:", matplotlib.__version__)
print("SITK version:", sitk.__version__)
print("Scikit-image version:", skimage.__version__)

Python version: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
Numpy version: 1.26.2
Matplotlib version: 3.8.2
SITK version: 2.3.1
Scikit-image version: 0.22.0


In [3]:
patient_id = 6

In [4]:
PREPROCESSED_BASE_DIR = '/mnt/data/datasets/RNSH_HFlung/pre-processed-plastimatch'
EVALUATION_DIR = '{}/evaluation'.format(PREPROCESSED_BASE_DIR)
PATIENT_EVALUATION_DIR = '{}/Patient{:02d}'.format(EVALUATION_DIR, patient_id)

#### read in the results XML and compare segmentation similarity

In [5]:
from bs4 import BeautifulSoup

In [6]:
CTVI_JAC_EVAL = '{}/seg-metrics-ctvi-jac.xml'.format(PATIENT_EVALUATION_DIR)
with open(CTVI_JAC_EVAL, 'r') as f:
    data = f.read()
ctvi_jac_eval = BeautifulSoup(data, "xml")

In [7]:
CTVI_NNUNET_EVAL = '{}/seg-metrics-ctvi-nnunet.xml'.format(PATIENT_EVALUATION_DIR)
with open(CTVI_NNUNET_EVAL, 'r') as f:
    data = f.read()
ctvi_nnunet_eval = BeautifulSoup(data, "xml")

In [8]:
patient_eval_d = {}

In [9]:
metrics_l = ['DICE','HDRFDST','MAHLNBS']
print('{: >10}{: >35}{: >15}{: >15}'.format('','','CTVI-Jac','CTVI-nnU-Net'))
for m in metrics_l:
    ctvi_jac_d = ctvi_jac_eval.find(m).attrs
    ctvi_nnunet_d = ctvi_nnunet_eval.find(m).attrs
    print('{: >10}{: >35}{: >15.2f}{: >15.2f}'.format(m, ctvi_jac_d['name'], float(ctvi_jac_d['value']), float(ctvi_nnunet_d['value'])))
    patient_eval_d[m] = {'ctvi_jac_d':ctvi_jac_d, 'ctvi_nnunet_d':ctvi_nnunet_d}

                                                    CTVI-Jac   CTVI-nnU-Net
      DICE      Dice Coefficient (F1-Measure)           0.72           0.69
   HDRFDST                 Hausdorff Distance          95.43          84.63
   MAHLNBS               Mahanabolis Distance           0.15           0.15


/home/daryl/miniconda3/envs/py310/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [12]:
DATASET_EVALUATION_FILE = '{}/evaluation.pkl'.format(EVALUATION_DIR)

with open(DATASET_EVALUATION_FILE, 'rb') as f:
    dataset_d = pickle.load(f)

dataset_d[patient_id]['segmentation-metrics'] = patient_eval_d

with open(DATASET_EVALUATION_FILE, 'wb') as f:
        pickle.dump(dataset_d, f)

In [14]:
dataset_d

{6: {'unquantised_spearman_correlation': 0.3804745064137437,
  'segmentation-metrics': {'DICE': {'ctvi_jac_d': {'name': 'Dice Coefficient (F1-Measure)',
     'symbol': 'DICE',
     'type': 'similarity',
     'value': '0.721899'},
    'ctvi_nnunet_d': {'name': 'Dice Coefficient (F1-Measure)',
     'symbol': 'DICE',
     'type': 'similarity',
     'value': '0.688932'}},
   'HDRFDST': {'ctvi_jac_d': {'name': 'Hausdorff Distance',
     'symbol': 'HDRFDST',
     'type': 'distance',
     'value': '95.425364',
     'executiontime': '1103',
     'unit': 'voxel'},
    'ctvi_nnunet_d': {'name': 'Hausdorff Distance',
     'symbol': 'HDRFDST',
     'type': 'distance',
     'value': '84.628600',
     'executiontime': '1133',
     'unit': 'voxel'}},
   'MAHLNBS': {'ctvi_jac_d': {'name': 'Mahanabolis Distance',
     'symbol': 'MAHLNBS',
     'type': 'distance',
     'value': '0.149659'},
    'ctvi_nnunet_d': {'name': 'Mahanabolis Distance',
     'symbol': 'MAHLNBS',
     'type': 'distance',
     'val